In [86]:
#!pip install pydrive
#!pip install googledrivedownloader
#!pip install yagmail

You should consider upgrading via the 'c:\software\anaconda3\python.exe -m pip install --upgrade pip' command.


In [222]:
# Load Packages

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os
import pandas as pd
import numpy as np
from datetime import datetime
import glob
import yagmail

os.chdir('C:\\File\\MCC Script - Python')

In [223]:
# Load Functions

def folder_retrieval(folder_id):
    query = "'" + folder_id + "'" + ' in parents and trashed = false'
    content_list = drive.ListFile({'q': query}).GetList()
    lst = []
    col = ['name','id','type']
    for content in content_list:
        lst.append([content['title'],content['id'],content['mimeType']])
    result = pd.DataFrame(lst, columns = col)
    return (result)

def file_reader(file_name):
    data = pd.read_excel(file_name,
                        sheet_name = 'Plan',
                        skiprows = 5,
                        index_col = None)
    try:
        data = data[data.Geo == 'US']
        data['Buy Details'] = file_name
    except: 
        data = 'Empty'
    return (data)

def file_combine(file_list):
    for i,each_file in enumerate(file_list):
        print('Combining ' + each_file)
        try:
            if i == 0:
                master = file_reader(each_file)
                cols = master.columns
            else:
                temp = file_reader(each_file)
                cols = temp.columns
                master = pd.concat([master, temp], ignore_index=True)
                master = master[cols]
        except:
            pass
    return(master)

def file_reader_cross_quarter(file_name):
    data = pd.read_excel(file_name,
                        sheet_name = 'Plan',
                        skiprows = 17,
                        index_col = None)
    try:
        data = data[data.Geo == 'US']
        data['Buy Details'] = file_name
    except: 
        data = 'Empty'
    return (data)

def file_combine_cross_quarter(file_list):
    for i,each_file in enumerate(file_list):
        print('Combining ' + each_file)
        try:
            if i == 0:
                master = file_reader_cross_quarter(each_file)
                cols = master.columns
            else:
                temp = file_reader(each_file)
                cols = temp.columns
                master = pd.concat([master, temp], ignore_index=True)
                master = master[cols]
        except:
            pass
    return(master)

In [224]:
# Authentication

gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=841426274923-leg1iqi48mjquh0j8h0qdgrqnsf81r4d.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [239]:
# Clear folder - Clorox

os.chdir('C:\\File\\Buy Details\\For Python\\AMJ')
files = os.listdir()
for f in files:
    os.remove(f)

In [250]:
## Step #1. Get id based on keyword search
    
query = "title contains '1. Buy Details' and mimeType = 'application/vnd.google-apps.folder'"
content_list = drive.ListFile({'q': query}).GetList()
for content in content_list:
    print(content['title'],content['id'])

# id = 'xxxxxxx'

## Step #2. Check what's inside the folder

query = "" + id + "in parents"
content_list = drive.ListFile({'q': query}).GetList()
for content in content_list:
    print(content['title'],content['id'])

1. Buy Details 15Iaop3OcnJaRsb7kvR979UczeC0R0sza
1. Buy Details 1BbTf8ckKpZEkNlS-IajXNGiaWXayOzQP
FY 19 Buy Details  1yaTtXUsdeke6CNBJrdZhfRkOArpILhOI


In [240]:
# Download BD - Clorox

Clorox_BD_ID = 'xxxxxxx'
Amazon_Cross_Quarter = 'xxxxxxx'

clorox_list = folder_retrieval(Clorox_BD_ID)
AMJ_ID = clorox_list[clorox_list.name == '4. AMJ FY20'].id.to_string(index = False)
AMJ_list = folder_retrieval(AMJ_ID)

Amazon_ID = clorox_list[clorox_list.name == 'Amazon'].id.to_string(index = False)
Amazon_list = folder_retrieval(Amazon_ID)
Amazon_AMJ = Amazon_list[Amazon_list.name == 'AMJ'].id.to_string(index = False)
Amazon_Cross_Quarter = Amazon_list[Amazon_list.name == 'Cross-Quarter'].id.to_string(index = False)
Amazon_AMJ_list = folder_retrieval(Amazon_AMJ)
Amazon_Cross_Quarter_list = folder_retrieval(Amazon_Cross_Quarter)

os.chdir('C:\\File\\Buy Details\\For Python\\AMJ')
File_Name = []
Brand = []
for i in range(len(AMJ_list)):
    brand_id = AMJ_list.id.iloc[i]
    files = folder_retrieval(brand_id)
    if len(files)>0:
        files = files[files.name != 'Archive']
        files = files[files.name != 'Archived']
        for j in range(len(files)):
            if files.name.iloc[j] != 'Untitled spreadsheet':
                File_Name.append(files.name.iloc[j])
                Brand.append(AMJ_list.name.iloc[i])
                file_id = files.id.iloc[j]
                temp = drive.CreateFile({'id': file_id})
                print('Downloading file:  %s' % files.name.iloc[j]) 
                temp.GetContentFile(files.name.iloc[j])

for i in range(len(Amazon_AMJ_list)):
    brand_id = Amazon_AMJ_list.id.iloc[i]
    files = folder_retrieval(brand_id)
    if len(files)>0:
        files = files[files.name != 'Archive']
        files = files[files.name != 'Archived']
        for j in range(len(files)):
            if files.name.iloc[j] != 'Untitled spreadsheet':
                File_Name.append(files.name.iloc[j])
                Brand.append('Amazon')
                file_id = files.id.iloc[j]
                temp = drive.CreateFile({'id': file_id})
                print('Downloading file:  %s' % files.name.iloc[j]) 
                temp.GetContentFile(files.name.iloc[j])

for i in range(len(Amazon_Cross_Quarter_list)):
    brand_id = Amazon_Cross_Quarter_list.id[i]
    files = folder_retrieval(brand_id)
    if len(files)>0:
        files = files[files.name.str.lower() != 'archive']
        files = files[files.name.str.lower() != 'archived']
        for j in range(len(files)):
            if files.name.iloc[j] != 'Untitled spreadsheet':
                if 'folder' in files.type.iloc[j]:
                    folder_id = files.id.iloc[j]
                    sub_files = folder_retrieval(folder_id)
                    if len(sub_files)>0:
                        sub_files = sub_files[sub_files.name.str.lower() != 'archive']
                        sub_files = sub_files[sub_files.name.str.lower() != 'archived']
                        for k in range(len(sub_files)):
                            if sub_files.name.iloc[k] != 'Untitled spreadsheet':
                                if 'folder' in sub_files.type.iloc[k]:
                                    folder_id_2 = sub_files.id.iloc[k]
                                    sub_files_2 = folder_retrieval(folder_id_2)
                                    if len(sub_files)>0:
                                        sub_files_2 = sub_files_2[sub_files_2.name.str.lower() != 'archive']
                                        sub_files_2 = sub_files_2[sub_files_2.name.str.lower() != 'archived']
                                        for l in range(len(sub_files_2)):
                                            File_Name.append(sub_files_2.name.iloc[l])
                                            Brand.append('Amazon Cross Quarter')
                                            sub_file_2_id = sub_files_2.id.iloc[l]
                                            temp = drive.CreateFile({'id': sub_file_2_id})
                                            print('Downloading file:  %s' % sub_files_2.name.iloc[l]) 
                                            temp.GetContentFile(sub_files_2.name.iloc[l])
                                else:
                                    File_Name.append(sub_files.name.iloc[k])
                                    Brand.append('Amazon Cross Quarter')
                                    sub_file_id = sub_files.id.iloc[k]
                                    temp = drive.CreateFile({'id': sub_file_id})
                                    print('Downloading file:  %s' % sub_files.name.iloc[k]) 
                                    temp.GetContentFile(sub_files.name.iloc[k])
                else:
                    File_Name.append(files.name.iloc[j])
                    Brand.append('Amazon Cross Quarter')
                    file_id = files.id.iloc[j]
                    temp = drive.CreateFile({'id': file_id})
                    print('Downloading file:  %s' % files.name.iloc[j]) 
                    temp.GetContentFile(files.name.iloc[j])
                    
Reference = pd.DataFrame({'file name': File_Name, 'brand': Brand})
Reference.to_csv('Reference.csv', index = False)

In [153]:
# Combine BD

os.chdir('C:\\File\\Buy Details\\For Python\\AMJ')
file_list = glob.glob('*.xlsm')
master = file_combine(file_list)

Combining AKQA_Clorox_FY20 AMJ_Media Plan_Buy Details_PPH_Portfolio_040720.xlsm
Combining AKQA_Clorox_FY20 AMJ_Media Plan_Buy Details_PSD DENTAL_040720.xlsm
Combining AKQA_Clorox_FY20 AMJ_Media Plan_Buy Details_PSD VET_040720.xlsm
Combining AKQA_Clorox_FY20_AMJ_Media Plan_Buy Details_Operating_Suite_040720.xlsm
Combining AKQA_Clorox_FY20_AMJ_Media Plan_Buy Details_PCF_040720.xlsm
Combining AKQA_Clorox_FY20_AMJ_Media Plan_Buy Details_PPH_Spore_Defense_052620.xlsm


C:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



Combining AKQA_Clorox_FY20_AMJ_Media Plan_Buy Details_TOT_Cleaning_040720.xlsm
Combining AKQA_Clorox_FY20_Media Plan_Buy Details_NAV NYNY Solo_JFM_2.19.xlsm
Combining AKQA_Clorox_FY20_Media Plan_Buy Details_NEO Solo_JFM_2.21.xlsm
Combining AKQA_Clorox_FY20_Media Plan_Buy Details_RNL NYNY Solo_JFM_12.20.xlsm
Combining BFR FY20 AMJ Brita Equity - Buy Details 051920.xlsm
Combining BHV_FY20 AMJ_Hidden Valley Condiments_Buy Details_4.20.2020.xlsm
Combining BNY FY20 AMJ Brita New Year New You - Buy Details 04.17.20.xlsm
Combining Burts Bees Mothers Day FY20 AMJ_6.1.20_incremental.xlsm
Combining Buy Details GTO FY20 JFM AMJ EVERGREEN .xlsm
Combining Buy Details Kingsford SY20 CH.2 Peak Season KFC GM 5.26.20.xlsm
Combining Buy Details Kingsford SY20 CH.3_KHW_5.26.20.xlsm
Combining Buy Details_BBD RNL RBL FY20 AMJ Prenatal Test_BD Updated.xlsm
Combining Buy details_BFG FY20 AMJ Brita Bottles_Final_4.24.20.xlsm
Combining Buy Details_BFR fy20 AMJ Creative Messaging Test .xlsm
Combining Buy detail

C:\Software\Anaconda3\lib\site-packages\pandas\core\ops.py:1164: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Combining Clorox Buy Details LCX FY20 AMJ C2_5.27.20.xlsm
Combining Clorox Buy Details Scentiva CST FY20 AMJ_052820.xlsm
Combining Clorox Buy Details Template_Mar_20Renew Life + VMS Mother's Day_AMJ_3.25.xlsm
Combining Clorox Buy Details_BBD FY20 AMJ Burt's Bees Fresh.xlsm
Combining Clorox Buy Details_CHC FY20 AMJ Clorox Homecare Franchise Fresh_Amazon_FINAL.xlsm
Combining Clorox Buy Details_FSF FY20 AMJ Fresh Step Fresh.xlsm
Combining Clorox Buy Details_HVF FY20 AMJ Fresh Amazon_v2.xlsm
Combining Clorox Buy Details_Mar_10_NAV FY20 AMJ Natural Vitality_052620.xlsm
Combining Clorox Buy Details_Mar_10_NEO FY20 AMJ NeoCell - GM_051220.xlsm
Combining Clorox Buy Details_Mar_10_RBL FY20 AMJ Rainbow Light_041720.xlsm
Combining Clorox Buy Details_Mar_20_BFC FY20 AMJ Burt's Bees Face Care_051420.xlsm
Combining Clorox Buy Details_Mar_20_FY20 AMJ Natural Vitality Solo_4.21.xlsm
Combining Clorox Buy Details_Mar_20_FY20 AMJ NeoCell Solo_5.18.xlsm
Combining Clorox Buy Details_Mar_20_FY20 AMJ Rainbow

In [154]:
# get brand name and BD for further mapping

reference = pd.read_csv('Reference.csv')
reference.rename(columns={'file name':'Buy Details', 'brand':'Brand'}, inplace=True)

In [155]:
# exclude empty rows and add brand info

master_copy = master
master = master[master['Geo'] != '']
master = master.merge(reference)

In [167]:
# Capture size of the placement

data = master
Size = []
for i in range(len(data)):
    if data['Placement Type'][i] == 'Package':
        Size.append('PKG')
    elif data['Placement Type'][i] == 'PKG':
        Size.append('PKG')
    elif ((data['Width'][i] == 'Vast') | (data['Width'][i] == 'VAST')):
        Size.append('0 x 0') 
    else:
        Size.append((str(data['Width'][i]) + ' x ' + str(data['Height'][i])).split('.')[0])
data['Size'] = Size

In [168]:
# Generate DCM placement name

columns_needed = data.loc[:,['Line Item', 'Geo','Site Name','audience + placement name','Vehicle','Cost Structure',
                            'Campaign ID','Inventory Source','Targetin WHO','Size','Site Served or Dart']]
data['DCM Placement Name'] = columns_needed.apply(lambda x: '|'.join(x.astype(str).values), axis=1)
data['DCM Placement Name'].replace({'\.0': ''}, inplace=True, regex=True)

In [169]:
# Prepare diagnosis columns

data['Note - Missing Campaign ID'] = ''
data['Note - Missing Placement ID'] = ''
data['Note - Incorrect Start Date'] = ''
data['Note - Missing Tag Type'] = ''
data['Note - Zero IO Rate'] = ''

In [170]:
# Flag rows with missing id

try:
    data.loc[data['DCM Placement ID'].isna(), ['Note - Missing Placement ID']] = 'x'
    data.loc[data['DCM Placement ID'].isna(), ['DCM Placement ID']] = 'Missing Placement ID'
except:
    print('Placment ID no problem!')
    
try:
    data.loc[data['Campaign ID'].isna(), ['Note - Missing Campaign ID']] = 'x'
    data.loc[data['Campaign ID'].isna(), ['Campaign ID']] = 'Missing Campaign ID'
except:
    print('Campaign ID no problem!')

In [171]:
# Flag rows with problematic start date: missing value, TBD, start date not falling in AMJ

try:
    data.loc[data['Start Date'] == 'TBD', ['Note - Incorrect Start Date']] = 'x'
except:
    print('No TBD found!')
try:
    data.loc[data['Start Date'].isna(), ['Note - Incorrect Start Date']] = 'x'
except:
    print('No missing value found!')
try:
    data.loc[(data['Start Date'] > datetime.strptime('2020-06-29', '%Y-%m-%d')) &
             (data['Brand'] != 'Amazon Cross Quarter'), ['Note - Incorrect Start Date']] = 'x'
    data.loc[(data['Start Date'] < datetime.strptime('2020-03-30', '%Y-%m-%d')) &
             (data['Brand'] != 'Amazon Cross Quarter'), ['Note - Incorrect Start Date']] = 'x'
except:
    print('start date all are proper!')

No TBD found!


In [172]:
# Flag rows for trackable placements with no tag type

data.loc[(data['Adserving Fees - Tag Type'] == '' ) & (data['Width'] != '2'), ['Note - Missing Tag Type']] = 'x'
data.loc[(data['Adserving Fees - Tag Type'] == '' ) & (data['Width'] != '2'), ['Adserving Fees - Tag Type']] = 'Missing Tag Type'

In [173]:
# Flag rows for non-VADD IO with zero or missing IO Rate

data.loc[(data['Net/Gross Rate'].isna()) & (data['Cost Structure'].str.lower() != 'vadd') & 
         (data['Cost Structure'].str.lower() != 'flat rate - impressions') & (data['Placement Type'] == 'Package'),['Note - Zero IO Rate']] = 'x'
data.loc[(data['Net/Gross Rate'] == 0) & (data['Cost Structure'].str.lower() != 'vadd') & 
         (data['Cost Structure'].str.lower() != 'flat rate - impressions') & (data['Placement Type'] == 'Package'),['Note - Zero IO Rate']] = 'x'
data.loc[(data['Net/Gross Rate'].isna()) & (data['Cost Structure'].str.lower() != 'vadd') & 
         (data['Cost Structure'].str.lower() != 'flat rate - impressions') & (data['Placement Type'] == 'Package'),['Net/Gross Rate']] = 'Zero IO Rate'
data.loc[(data['Net/Gross Rate'] == 0) & (data['Cost Structure'].str.lower() != 'vadd') & 
         (data['Cost Structure'].str.lower() != 'flat rate - impressions') & (data['Placement Type'] == 'Package'),['Net/Gross Rate']] = 'Zero IO Rate'

In [174]:
# Convert CPE to CPA

data.loc[data['Cost Structure'] == 'CPE', 'Cost Structure'] = 'CPA'

In [175]:
# Generate problematic file

problematic_rows = data.loc[(data['Note - Missing Campaign ID'] == 'x') | 
                            (data['Note - Missing Placement ID'] == 'x') |
                            (data['Note - Incorrect Start Date'] == 'x') |
                            (data['Note - Missing Tag Type'] == 'x') |
                            (data['Note - Zero IO Rate'] == 'x')]

In [176]:
# Output result

os.chdir('C:\\File\\Buy Details\\For Python\\AMJ - Output')
files = os.listdir()
for f in files:
    os.remove(f)
problematic_rows.to_csv('Clorox AMJ Problematic File.csv', index = False)
data.to_csv('Clorox AMJ BD Master File.csv', index = False)

In [253]:
# Sending Emails
# Turn on Less Secure App Access:
# https://myaccount.google.com/lesssecureapps

yag = yagmail.SMTP('xxxxxxx', 'xxxxxxx')

In [255]:
# Sending Problematic File

send_to = ['someone@xx.com']
problematic_summary = problematic_rows.loc[:,['Buy Details']]
contents = [
    "Below are the BDs with problems, more info attached:", 
    problematic_summary.drop_duplicates().to_html(),
    "C:\\File\\Buy Details\\For Python\\AMJ - Output\\Clorox AMJ Problematic File.csv"
]
yag.send(send_to, 'test', contents)

{}